In [11]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
from tensorflow.keras import datasets, layers, models

In [12]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

In [13]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0] - 1

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0] - 1  

In [14]:
imgs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs.append(img)
print("--- %s seconds ---" % (time.time() - start_time))  

imgs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs_test.append(img)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 77.71262311935425 seconds ---
--- 20.534557104110718 seconds ---


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(188, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(np.array(imgs), emotion_idx, epochs=10, validation_data=(np.array(imgs_test), test_emotion_idx))

Train on 1999 samples, validate on 501 samples
Epoch 1/10
1984/1999 [============================>.] - ETA: 3s - loss: 3.1359 - acc: 0.0449 